In [4]:
import torch
import numpy as np
import pandas as pd
import os

rdir = '/data/hyeryung'

if os.getcwd() != f'{rdir}/mucoco':
    print(f"change dir to '{rdir}/mucoco'")
    os.chdir(f'{rdir}/mucoco')

from notebooks.utils.load_ckpt import define_model

In [5]:
test_all = pd.read_json('data/toxicity/jigsaw-unintended-bias-in-toxicity-classification/fine-grained/test.jsonl',
                    lines=True)

In [6]:
print("num test samples", test_all.shape[0])

num test samples 3998


In [7]:
print(f"checking duplicates... {test_all.duplicated(subset=['text']).sum()} duplicated texts")

checking duplicates... 0 duplicated texts


In [8]:
test = test_all.loc[test_all['toxicity']>=0.5].copy()

In [9]:
print("num toxic test samples", test.shape[0])

num toxic test samples 1960


In [10]:
# check distribution by toxicity
test = test.reset_index(drop=True)
test['toxicity_bin'] = pd.cut(test['toxicity'], np.arange(0.5, 1.1, 0.1), right=False)#, labels=["[0.5, 0.6)"])
test_=test.copy()
test.groupby('toxicity_bin').size()

toxicity_bin
[0.5, 0.6)    659
[0.6, 0.7)    498
[0.7, 0.8)    397
[0.8, 0.9)    300
[0.9, 1.0)     51
[1.0, 1.1)     55
dtype: int64

In [11]:
# 어떻게 하는게 가장 좋을까? 흠.. 
# 고려해야 할점.. 
# - 어떤 UI 가 라벨링하기 좋을까? -> 일단 그것부터 생각?
# - toxicity 값은 보여줄까? -> 일단 샘플을 다 뽑아놓고, 필터링해서 제공하면 되지 않을까? 일단 컬럼에 포함.

In [12]:
# 교수님과 얘기했던 UI = 일단은 word 단위로 라벨을 받아보자.
test['text_words'] = test.text.str.split()

In [13]:
test

,text,toxicity,toxicity_bin,text_words
0,You aren't that bright. Citizens were picking ...,0.500000,"[0.5, 0.6)","[You, aren't, that, bright., Citizens, were, p..."
1,References are constantly being made that ind...,0.800000,"[0.8, 0.9)","[References, are, constantly, being, made, tha..."
2,"""...these are the people.."" Those REAL people...",0.833333,"[0.8, 0.9)","[""...these, are, the, people.."", Those, REAL, ..."
3,This article is especially hilarious in light ...,0.600000,"[0.6, 0.7)","[This, article, is, especially, hilarious, in,..."
4,Absolutely not! We should never learn to live...,0.500000,"[0.5, 0.6)","[Absolutely, not!, We, should, never, learn, t..."
...,...,...,...,...
1955,I don’t see it happening because of all the pr...,0.750000,"[0.7, 0.8)","[I, don’t, see, it, happening, because, of, al..."
1956,Every time a lib opens his mouth more garbage ...,0.600000,"[0.6, 0.7)","[Every, time, a, lib, opens, his, mouth, more,..."
1957,the dealers don't have a hard time making mone...,0.738462,"[0.7, 0.8)","[the, dealers, don't, have, a, hard, time, mak..."
1958,My opinion is Trump keeps opening his mouth an...,0.600000,"[0.6, 0.7)","[My, opinion, is, Trump, keeps, opening, his, ..."


In [14]:
words = pd.DataFrame(test['text_words'].tolist())

In [15]:
lens = pd.DataFrame(test['text_words'].apply(len)).rename(columns={'text_words': 'num_words'})

In [16]:
df_for_labelling = pd.concat([test[['text', 'toxicity']], lens, words], axis=1)

In [18]:
df_for_labelling.to_excel('notebooks/locate/labelling/test_geq_0.5.xlsx')

In [81]:
df_for_labelling = pd.read_excel('notebooks/locate/labelling/test_geq_0.5.xlsx', index_col=0)

In [82]:
df_for_labelling["type"] = 0

In [83]:
df_for_labelling["text_index"] = df_for_labelling.index

In [84]:
blanks_for_labelling = df_for_labelling.copy()
blanks_for_labelling["type"] = 1
blanks_for_labelling.loc[:, 0:195] = None

In [85]:
df_for_labelling = pd.concat([df_for_labelling, blanks_for_labelling], axis=0)

In [86]:
df_for_labelling = df_for_labelling.sort_values(by=['text_index', 'type'], ascending=[True, True])

In [87]:
df_for_labelling = df_for_labelling.reset_index(drop=True)

In [88]:
df_for_labelling['type'].replace({0: 'text', 1: 'label'}, inplace=True)

In [90]:
df_for_labelling.columns

Index([      'text',   'toxicity',  'num_words',            0,            1,
                  2,            3,            4,            5,            6,
       ...
                188,          189,          190,          191,          192,
                193,          194,          195,       'type', 'text_index'],
      dtype='object', length=201)

In [91]:
df_for_labelling.to_excel('notebooks/locate/labelling/test_geq_0.5_v0.2.xlsx')

### Old code (06.08)

In [ ]:
test = pd.read_json('data/toxicity/jigsaw-unintended-bias-in-toxicity-classification/fine-grained/test.jsonl',
                    lines=True)

In [ ]:
test = test.loc[test['toxicity']>=0.5].copy()

In [3]:
## select toxic 0.5~1.0
test = test.reset_index(drop=True)
test['toxicity_bin'] = pd.cut(test['toxicity'], np.arange(0.5, 1.1, 0.1), labels=np.arange(6))
test_=test.copy()

## randomly select from each bin
tot_count=150
bin_count=6
test = test.groupby('toxicity_bin').sample(int(tot_count/bin_count), random_state=11)

## add 4 examples from toxicity 1
test = pd.concat([test, test_.loc[((~test_.index.isin(test.index)) & (test_.toxicity>0.99))].sample(4, random_state=11)], axis=0)

test.to_json('./notebooks/results/labelling_samples_test_geq0.5_150.jsonl', orient='records', lines=True)

In [6]:
model,config,tokenizer=define_model('/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin',
                                 output_attentions=True, output_hidden_states=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

DEVICE:  cuda


In [7]:
tokenized=tokenizer(test['text'].tolist(),padding=True, return_tensors="pt", truncation=True)

In [8]:
config.max_position_embeddings

514

In [9]:
# model input size와 같아야할거같은데!
tokenized_ids=tokenized['input_ids']

In [10]:
res = []
for i in range(tokenized_ids.shape[0]):
    tmp_sent = tokenized_ids[i]
    tmp_toks = [tokenizer.decode(x) for x in tmp_sent]
    res.append(tmp_toks)

In [11]:
if os.path.exists('./notebooks/results/toxic_labels.txt'):
    with open('./notebooks/results/toxic_labels.txt','r') as f:
        contents = f.readlines()
    last_index = len(contents)/2
    print(f'starting from index {last_index}')
else:
    fptr = open('./notebooks/results/toxic_labels.txt', 'w')
    fptr.close()
    last_index=0

starting from index 4.0


In [49]:
for i in range(last_index, tokenized_ids.shape[0]):
    try:
        print(f"{test['toxicity'].tolist()[i]} ||||||||||| {''.join(res[i])}")
        for j in range(len(res[i])):
            if res[i][j] != '<|endoftext|>':
                print(f"{j} ------ {res[i][j]}")
        toxic_tokens=input("input comma separated list of tokens that are toxic >>")
        if '-' in toxic_tokens:
            start, end = toxic_tokens.split('-')    
            toxic_tokens = ','.join([str(x) for x in range(int(start.strip()), int(end.strip())+1,1)])
        print(f"List you typed: {toxic_tokens}")
        # toxic_tokens=toxic_tokens.split(",")
        fptr = open('./notebooks/results/toxic_labels.txt', 'a')
        fptr.write(''.join(res[i]) + '\n')
        fptr.write(toxic_tokens + '\n')
        fptr.close()
    except KeyboardInterrupt:
        fptr = open('./notebooks/results/toxic_labels.txt', 'a')
        fptr.write(i + '\n')
        fptr.close()
        last_index = i    
    

0.525 ||||||||||| as long as you cannot admit who your enemies are you are losing fast. these guys want you dead yet they get modern missiles from your army. WTF!!!!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e